In [19]:
import rectools
from rectools.dataset import Dataset
from rectools.columns import Columns
import pandas as pd
import numpy as np
from random import randint

In [11]:
data = pd.read_csv("C:/Users/elise/Downloads/rec_train.csv", 
                 header=None,
                 names=[Columns.User, Columns.Item])

In [41]:
data[Columns.Item].value_counts()

item_id
7      519
1      510
15     498
34     322
3      304
      ... 
206      1
120      1
154      1
26       1
183      1
Name: count, Length: 172, dtype: int64

In [13]:
df = data.iloc[1:]

In [18]:
df[Columns.Weight] = 1

C:\Users\elise\AppData\Local\Temp\ipykernel_6792\3826501316.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[Columns.Weight] = 1


In [20]:
df[Columns.Datetime] = 52

C:\Users\elise\AppData\Local\Temp\ipykernel_6792\2651942712.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[Columns.Datetime] = 52


In [22]:
dataset = Dataset.construct(df)

In [24]:
from rectools.models.lightfm import LightFMWrapperModel
from rectools.models.lightfm import LightFM

In [48]:
lightfm_model = LightFM()
model = LightFMWrapperModel(lightfm_model, epochs=100, num_threads=16, verbose=1)

In [49]:
model.fit(dataset)

Epoch: 100%|████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 127.28it/s]


In [50]:
submission1 = pd.read_csv('C:/Users/elise/Downloads/rec_submission.csv',
                         header=None,
                         names=[Columns.User, 'course_id_1', 'course_id_2', 'course_id_3'])
submission1 = submission1.iloc[1:]
submission1

,user_id,course_id_1,course_id_2,course_id_3
1,78,0,0,0
2,81,0,0,0
3,120,0,0,0
4,123,0,0,0
5,150,0,0,0
...,...,...,...,...
2896,185864,0,0,0
2897,186262,0,0,0
2898,186691,0,0,0
2899,186887,0,0,0


In [51]:
recs = model.recommend(users=submission1[Columns.User].unique(),
                      dataset=dataset, 
                      k=3,
                      filter_viewed=True)

In [52]:
recs.head(10)

,user_id,item_id,score,rank
0,78,7,5.935275,1
1,78,1,5.933095,2
2,78,15,5.893421,3
3,81,7,5.963667,1
4,81,15,5.932183,2
5,81,3,5.448052,3
6,120,7,5.848666,1
7,120,15,5.821408,2
8,120,34,5.404459,3
9,150,7,5.854123,1


In [53]:
preds = []
k = 0
lst = []
for x in recs[Columns.Item]:
    if len(lst) == 3:
        preds.append(lst)
        lst = []
    lst.append(int(x))
preds[0:4]

[[7, 1, 15], [7, 15, 3], [7, 15, 34], [7, 1, 15]]

In [54]:
submission = pd.read_csv('C:/Users/elise/Downloads/rec_submission.csv')
submission.head()

,user_id,course_id_1,course_id_2,course_id_3
0,78,0,0,0
1,81,0,0,0
2,120,0,0,0
3,123,0,0,0
4,150,0,0,0


In [55]:
for i, x in enumerate(preds):
    c1, c2, c3 = x
    submission.iloc[i, 1] = c1
    submission.iloc[i, 2] = c2
    submission.iloc[i, 3] = c3

In [56]:
submission.head()

,user_id,course_id_1,course_id_2,course_id_3
0,78,7,1,15
1,81,7,15,3
2,120,7,15,34
3,123,7,1,15
4,150,7,1,15


In [57]:
submission.to_csv('C:/Users/elise/Downloads/submission_r1.csv', index=False)